In [1]:
from time import perf_counter
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from joblib import Parallel, delayed
import os
import json
import dag_utils as utils
from Baselines import Nonneg_dagma, MetMulDagma
from TopoGreedy import BUILD
from Baselines import colide_ev
from Baselines import DAGMA_linear
from Baselines import notears_linear


PATH = './results/samples/'
SAVE = True 
SEED = 10
N_CPUS = os.cpu_count()  # Get number of available CPUs
np.random.seed(SEED)

In [2]:
def _to_jsonable(obj):
    """Recursively convert objects to JSON-serializable forms."""
    import numpy as np
    from pathlib import Path

    # basic types
    if obj is None or isinstance(obj, (bool, int, float, str)):
        # cast numpy scalars to Python
        if isinstance(obj, (np.bool_, np.integer, np.floating)):
            return obj.item()
        return obj

    # numpy arrays -> lists (careful for huge arrays: we won't dump arrays here)
    if isinstance(obj, np.ndarray):
        return obj.tolist()

    # tuples -> lists
    if isinstance(obj, tuple):
        return [_to_jsonable(x) for x in obj]

    # lists
    if isinstance(obj, list):
        return [_to_jsonable(x) for x in obj]

    # dicts
    if isinstance(obj, dict):
        return {str(k): _to_jsonable(v) for k, v in obj.items()}

    # Path
    if isinstance(obj, Path):
        return str(obj)

    # dataclasses
    try:
        from dataclasses import is_dataclass, asdict
        if is_dataclass(obj):
            return _to_jsonable(asdict(obj))
    except Exception:
        pass

    # callables (functions/classes)
    if callable(obj):
        # try to capture module + name; fall back to repr
        name = getattr(obj, "__name__", obj.__class__.__name__)
        mod  = getattr(obj, "__module__", None)
        return f"{mod+'.' if mod else ''}{name}"

    # objects with __dict__
    if hasattr(obj, "__dict__"):
        return _to_jsonable(vars(obj))

    # fallback
    return repr(obj)


In [3]:
# -*- coding: utf-8 -*-
"""
Structured DAG experiments: data simulation -> baselines -> metrics -> save & return.

Assumes the following are available in scope or importable:
  - utils.simulate_sem, utils.standarize, utils.to_bin, utils.count_accuracy, utils.compute_norm_sq_err
  - TopoGreedy_refresh, DAGMA_linear, colide_ev, notears_linear
"""

from __future__ import annotations
import os, json, math, uuid, shutil, datetime as dt
from dataclasses import dataclass, asdict, field
from pathlib import Path
from time import perf_counter
from typing import Callable, Dict, Any, List, Optional, Tuple

import numpy as np
from sklearn.metrics import f1_score
from Baselines import GOLEM_Torch
import matplotlib.pyplot as plt

# ---------------------------------------------
# Utility: lambda schedule
# ---------------------------------------------
def get_lambda_value(n_nodes: int, n_samples: int, times: float = 1.0) -> float:
    """Common λ heuristic: sqrt(log p / n) scaled by `times`."""
    return math.sqrt(max(1e-12, np.log(max(2, n_nodes))) / max(2, n_samples)) * times


# ---------------------------------------------
# Baseline spec & registry
# ---------------------------------------------
@dataclass
class BaselineSpec:
    """
    Unified baseline configuration.

    - model: either a *callable estimator* with .fit() method (class) OR a function (e.g., notears_linear)
    - init: kwargs passed to class constructor (ignored if model is a function)
    - args: kwargs passed to .fit(...) or function call
    - name: label for legends / saving
    - standardize: if True, pass standardized X to the model; else pass raw X
    - adapt_lambda: if True, rescale 'lamb' or 'lambda1' in args per (p, n)
    - topo_transpose: if True, transpose W_est and binarized variant to align with your downstream expectations
                      (e.g., TopoGreedy variants that output transposed)
    - is_topogreedy_refresh: if True, expects dict output with keys {"prec","A_est","A_est_bin"}; handled specially
    """
    model: Any
    init: Dict[str, Any] = field(default_factory=dict)
    args: Dict[str, Any] = field(default_factory=dict)
    name: str = "baseline"
    standardize: bool = False
    adapt_lambda: bool = False
    topo_transpose: bool = False
    is_topogreedy_refresh: bool = False


# ---------------------------------------------
# Experiment runner
# ---------------------------------------------
@dataclass
class ExperimentConfig:
    n_graphs: int
    n_nodes: int
    n_samples_list: List[int]
    edge_threshold: float
    data_params: Dict[str, Any]
    baselines: List[BaselineSpec]
    out_dir: str = "./exp_results"
    run_tag: Optional[str] = None
    save_intermediate: bool = True
    seed_offset: int = 0  # to vary graph seeds across runs


class ExperimentRunner:
    def __init__(self, cfg: ExperimentConfig):
        self.cfg = cfg
        self.run_id = cfg.run_tag or dt.datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:6]
        self.out_root = Path(cfg.out_dir) / self.run_id
        self.out_root.mkdir(parents=True, exist_ok=True)
        self._save_manifest()

    # ---------- public API ----------
    def run(self, verbose: bool = True):
        """
        Orchestrates:
          - for each graph (with new simulation seed),
          - for each n_samples in list,
          - for each baseline,
            => train / estimate => metrics => save.

        Returns a dict of stacked arrays ready-to-plot.
        """
        B = len(self.cfg.baselines)
        S = len(self.cfg.n_samples_list)
        N = self.cfg.n_nodes
        G = self.cfg.n_graphs

        # Metrics tensors: (G, S, B)
        shd = np.zeros((G, S, B))
        tpr = np.zeros((G, S, B))
        fdr = np.zeros((G, S, B))
        fscore = np.zeros((G, S, B))
        err = np.zeros((G, S, B))
        runtime = np.zeros((G, S, B))
        dag_count = np.zeros((G, S, B))
        # Frobenius diff(Theta): normalized (G, S, B)
        theta_diff = np.zeros((G, S, B))

        # Store full adjacency estimates: (G, S, B, N, N)
        W_est_all = np.zeros((G, S, B, N, N))
        # Optional precision matrices when available (fill zeros otherwise)
        Theta_est_all = np.zeros((G, S, B, N, N))

        for g in range(G):
            if verbose:
                print(f"\n=== Graph {g+1}/{G} ===")

            # -------------------- simulate data for this graph --------------------
            graph_seed = self.cfg.seed_offset + g
            data_p = dict(self.cfg.data_params)
            data_p["n_nodes"] = self.cfg.n_nodes

            # we will re-simulate for each n_samples (fresh X but same W_true structure per spec)
            W_true_cache = None
            Theta_true_cache = None

            for si, n_samples in enumerate(self.cfg.n_samples_list):
                data_p_this = dict(data_p)
                data_p_this["n_samples"] = int(n_samples)

                # simulate SEM
                W_true, _, X, Theta_true = utils.simulate_sem(**data_p_this)

                # cache structure (if consistent across n_samples) — harmless if overwritten
                W_true_cache = W_true
                Theta_true_cache = Theta_true

                X_std = utils.standarize(X) if data_p_this.get("standarize", False) else X
                W_true_bin = utils.to_bin(W_true, self.cfg.edge_threshold)
                norm_W_true = np.linalg.norm(W_true)
                # emp_cov = (X_std.T @ X_std) / float(X_std.shape[0])
                emp_cov = np.cov(X_std, rowvar = False)
                print(f"cond: {np.linalg.cond(Theta_true)}")
                if verbose:
                    print(f"- samples={n_samples}, edges≈{np.count_nonzero(W_true_bin)}")

                # -------------------- run all baselines --------------------
                for bi, base in enumerate(self.cfg.baselines):
                    X_in = X_std if base.standardize else X
                    # args per baseline (copy so we can mutate lambdas safely)
                    args_call = dict(base.args)

                    # adaptive λ scheduling
                    if base.adapt_lambda:
                        if "lamb" in args_call:
                            args_call["lamb"] = get_lambda_value(self.cfg.n_nodes, n_samples, args_call["lamb"])
                        if "lambda1" in args_call:
                            args_call["lambda1"] = get_lambda_value(self.cfg.n_nodes, n_samples, args_call["lambda1"])

                    # ---------- execute baseline ----------
                    t0 = perf_counter()
                    W_est, Theta_est = self._run_one_baseline(
                        base=base,
                        X=X_in,
                        X_std=X_std,
                        emp_cov=emp_cov,
                        edge_thr=self.cfg.edge_threshold
                    )
                    t1 = perf_counter()
                    

                    # NaN safeguard
                    if np.isnan(W_est).any():
                        W_est = np.zeros_like(W_est)
                        W_bin = np.zeros_like(W_est)
                    else:
                        W_bin = utils.to_bin(W_est, self.cfg.edge_threshold)

                    # Optional orientation fix for TopoGreedy-like outputs
                    if base.topo_transpose:
                        W_est = W_est.T
                        W_bin = W_bin.T

                    # ---------- metrics ----------
                    shd[g, si, bi], tpr[g, si, bi], fdr[g, si, bi] = utils.count_accuracy(W_true_bin, W_bin)
                    fscore[g, si, bi] = f1_score(W_true_bin.flatten(), W_bin.flatten())
                    err[g, si, bi] = utils.compute_norm_sq_err(W_true, W_est, norm_W_true)
                    runtime[g, si, bi] = (t1 - t0)
                    dag_count[g, si, bi] = 1.0 if utils.is_dag(W_bin) else 0.0

                    # Theta Frobenius difference (if Theta_est available)
                    if Theta_est is None:
                        theta_diff[g, si, bi] = 0.0
                    else:
                        # num = np.linalg.norm(Theta_est - Theta_true, "fro")
                        # den = np.linalg.norm(Theta_true, "fro") + 1e-8
                        Theta_norm = np.linalg.norm(Theta_true, "fro")
                        theta_diff[g, si, bi] = utils.compute_norm_sq_err(Theta_true, Theta_est, Theta_norm)

                    # store estimates
                    W_est_all[g, si, bi] = W_est
                    if Theta_est is not None:
                        Theta_est_all[g, si, bi] = Theta_est

                    if verbose:
                        print(f"  · {base.name:<18s} | SHD {shd[g,si,bi]:.1f} | F1 {fscore[g,si,bi]:.3f} | "
                              f"ΘΔ {theta_diff[g,si,bi]:.3f} | {runtime[g,si,bi]:.2f}s")


                    # print(f"W_est: {W_est}")
                # optional per-(graph,samples) checkpoint save
                if self.cfg.save_intermediate:
                    data = self._save_block(
                        g=g, si=si,
                        W_true=W_true, Theta_true=Theta_true,
                        W_est_all=W_est_all[g, si],
                        Theta_est_all=Theta_est_all[g, si],
                        shd=shd[g, si], tpr=tpr[g, si], fdr=fdr[g, si],
                        f1=fscore[g, si], err=err[g, si], rt=runtime[g, si],
                        dags=dag_count[g, si], theta_diff=theta_diff[g, si]
                    )

        # ------------- stack & save final -------------
        final = dict(
            shd=shd, tpr=tpr, fdr=fdr, f1=fscore, err=err,
            runtime=runtime, dag_count=dag_count, theta_diff=theta_diff,
            W_est_all=W_est_all, Theta_est_all=Theta_est_all
        )
        np.savez_compressed(self.out_root / "final_results.npz", **final)
        if verbose:
            print(f"\nSaved results to: {self.out_root}")

        return final, data

    # ---------- internals ----------
    def _run_one_baseline(
        self,
        base: BaselineSpec,
        X: np.ndarray,
        X_std: np.ndarray,
        emp_cov: np.ndarray,
        edge_thr: float
    ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """
        Runs a single baseline and returns (W_est, Theta_est or None).

        Special handling:
          - function baselines (e.g., notears_linear): W_est = f(X, **args)
          - TopoGreedy_refresh: expects dict with {'A_est','A_est_bin','prec'}
          - class baselines: model = cls(**init); model.fit(X, **args); W_est = model.W_est
            If model exposes Theta via .Theta_est (or .prec), we capture it.
        """
        # function-style baseline (no .fit)
        if callable(base.model) and not hasattr(base.model, "fit"):
            # handle TopoGreedy_refresh function signature:
            if base.is_topogreedy_refresh:
                out = base.model(X, emp_cov, **base.args)
                W_est = out.get("A_est", None)
                Theta_est = out.get("prec", None)
                if W_est is None:
                    W_est = np.zeros((X.shape[1], X.shape[1]))
                return W_est, Theta_est
            else:
                # generic function baseline like notears_linear(X, **args)
                W_est = base.model(X, **base.args)
                return W_est, None

        # class-style baseline
        model = base.model(**base.init) if base.init else base.model()
        # Try to pass standardized X only if the baseline asked for it (already handled by caller via X argument)
        model.fit(X, **base.args)

        # Extract adjacency
        W_est = getattr(model, "W_est", None)
        if W_est is None:
            # Some models might return directly from fit (rare); fallback to zeros
            W_est = np.zeros((X.shape[1], X.shape[1]))

        # Extract precision if present
        Theta_est = None
        if hasattr(model, "Theta_est"):
            Theta_est = getattr(model, "Theta_est")
        elif hasattr(model, "prec"):
            Theta_est = getattr(model, "prec")

        # TopoGreedy (non-refresh) variants sometimes return A transposed relative to convention;
        # caller can request topo_transpose=True in BaselineSpec to fix orientation.
        return W_est, Theta_est

    def _save_manifest(self):
        """Save config manifest (JSON) for reproducibility, without non-serializable fields."""
        # build a sanitized copy of the config:
        #   - convert baselines to lightweight dicts (stringify the model)
        sanitized_cfg = {
            "n_graphs": self.cfg.n_graphs,
            "n_nodes": self.cfg.n_nodes,
            "n_samples_list": list(self.cfg.n_samples_list),
            "edge_threshold": self.cfg.edge_threshold,
            "data_params": _to_jsonable(self.cfg.data_params),
            "baselines": [
                {
                    "name": b.name,
                    "model": _to_jsonable(b.model),           # "module.ClassOrFunc"
                    "init": _to_jsonable(b.init),
                    "args": _to_jsonable(b.args),
                    "standardize": b.standardize,
                    "adapt_lambda": b.adapt_lambda,
                    "topo_transpose": b.topo_transpose,
                    "is_topogreedy_refresh": b.is_topogreedy_refresh,
                }
                for b in self.cfg.baselines
            ],
            "out_dir": str(self.cfg.out_dir),
            "run_tag": self.cfg.run_tag,
            "save_intermediate": self.cfg.save_intermediate,
            "seed_offset": self.cfg.seed_offset,
        }

        manifest = {
            "run_id": self.run_id,
            "created_at": dt.datetime.now().isoformat(),
            "config": sanitized_cfg,
            "python": {"numpy_version": np.__version__},
        }

        (self.out_root / "config.json").write_text(json.dumps(_to_jsonable(manifest), indent=2))

    def _save_block(
        self, g: int, si: int,
        W_true: np.ndarray, Theta_true: np.ndarray,
        W_est_all: np.ndarray,
        Theta_est_all: np.ndarray,
        shd: np.ndarray, tpr: np.ndarray, fdr: np.ndarray,
        f1: np.ndarray, err: np.ndarray, rt: np.ndarray,
        dags: np.ndarray, theta_diff: np.ndarray
    ):
        """Persist per-(graph, sample-index) blob for debugging/inspection."""
        sub = self.out_root / f"graph_{g:03d}" / f"samples_{self.cfg.n_samples_list[si]}"
        sub.mkdir(parents=True, exist_ok=True)
        filename = sub / "block.npz"
        np.savez_compressed(
            filename,
            W_true=W_true, Theta_true=Theta_true,
            W_est_all=W_est_all, Theta_est_all=Theta_est_all,
            shd=shd, tpr=tpr, fdr=fdr, f1=f1, err=err, runtime=rt,
            dag_count=dags, theta_diff=theta_diff
        )
        
        # Return the complete data as a dictionary containing all arrays
        return {
            'filename': str(filename),
            'W_true': W_true,
            'Theta_true': Theta_true,
            'W_est_all': W_est_all,
            'Theta_est_all': Theta_est_all,
            'shd': shd,
            'tpr': tpr,
            'fdr': fdr,
            'f1': f1,
            'err': err,
            'runtime': rt,
            'dag_count': dags,
            'theta_diff': theta_diff
        }



In [4]:
# ---------------------------------------------
# Plotting helpers
# ---------------------------------------------
def plot_summary_curves(final: Dict[str, np.ndarray],
                        cfg: ExperimentConfig,
                        out_dir: Path,
                        baselines: List[BaselineSpec]):
    """
    final: output of runner.run(); contains arrays (G,S,B,...)
    Plots mean ± s.e.m. across G graphs, vs n_samples, for each baseline.
    Also adds shaded regions for 10th-90th percentiles.
    Saves PNGs to out_dir.
    """
    n_samples = np.array(cfg.n_samples_list)
    G = final["shd"].shape[0]
    B = final["shd"].shape[2]

    # --- means & standard errors over graphs (axis=0) ---
    shd_mean = final["shd"].mean(axis=0)                        # (S, B)
    shd_sem  = final["shd"].std(axis=0, ddof=1) / np.sqrt(G)    # (S, B)
    shd_p10  = np.percentile(final["shd"], 10, axis=0)          # (S, B)
    shd_p90  = np.percentile(final["shd"], 90, axis=0)          # (S, B)

    err_mean = final["err"].mean(axis=0)                        # (S, B)
    err_sem  = final["err"].std(axis=0, ddof=1) / np.sqrt(G)    # (S, B)
    err_p10  = np.percentile(final["err"], 10, axis=0)          # (S, B)
    err_p90  = np.percentile(final["err"], 90, axis=0)          # (S, B)

    # --- SHD vs #samples ---
    plt.figure()
    for bi in range(B):
        plt.errorbar(
            n_samples, shd_mean[:, bi], yerr=shd_sem[:, bi],
            marker='o', capsize=4, linewidth=2, label=baselines[bi].name
        )
        plt.fill_between(
            n_samples, shd_p10[:, bi], shd_p90[:, bi],
            alpha=0.2
        )
    plt.xlabel("Number of samples (n)")
    plt.ylabel("SHD (mean ± s.e.m. over graphs)")
    plt.title("SHD vs #Samples")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "shd_vs_samples.png", dpi=200)

    # --- Normalized MSE (weights) vs #samples ---
    plt.figure()
    for bi in range(B):
        plt.errorbar(
            n_samples, err_mean[:, bi], yerr=err_sem[:, bi],
            marker='o', capsize=4, linewidth=2, label=baselines[bi].name
        )
        plt.fill_between(
            n_samples, err_p10[:, bi], err_p90[:, bi],
            alpha=0.2
        )
    plt.xlabel("Number of samples (n)")
    plt.ylabel("Normalized MSE of weights (mean ± s.e.m.)")
    plt.title("Normalized MSE (weights) vs #Samples")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "nmse_vs_samples.png", dpi=200)

    print(f"Saved plots to: {out_dir}")


In [5]:
# Load and examine the npz file
import numpy as np

# data = np.load('/Users/hamedajorlou/Documents/GROW/exp_results/20250917_004018_f9a912/graph_000/samples_1000/block.npz')


import re

def _sanitize_baseline_name_for_csv(name: str) -> str:
    """Turn 'TopoGreedy-0.02' -> 'TopoGreedy_0_02' (safe CSV column header)."""
    return re.sub(r'[^A-Za-z0-9]+', '_', name).strip('_')

def save_summary_csvs(final: Dict[str, np.ndarray],
                      cfg: ExperimentConfig,
                      out_dir: Path,
                      baselines: List[BaselineSpec]) -> Tuple[Path, Path, Path]:
    """
    Writes three CSVs into out_dir:
      - shd_vs_samples.csv:   n_samples + (mean, sem, p10, p90) per baseline
      - nmse_vs_samples.csv:  n_samples + (mean, sem, p10, p90) per baseline
      - summary_vs_samples.csv: wide table with both metrics interleaved
    Returns the three paths.
    """
    import csv
    n_samples = np.asarray(cfg.n_samples_list)
    G = int(final["shd"].shape[0])
    B = int(final["shd"].shape[2])

    # Means, SEM, and percentiles across graphs
    def _sem(arr):
        return (arr.std(axis=0, ddof=1) / np.sqrt(G)) if G > 1 else np.zeros_like(arr.mean(axis=0))

    shd_mean = final["shd"].mean(axis=0)   # (S,B)
    shd_sem  = _sem(final["shd"])          # (S,B)
    shd_p10  = np.percentile(final["shd"], 10, axis=0)  # (S,B)
    shd_p90  = np.percentile(final["shd"], 90, axis=0)  # (S,B)
    
    nmse_mean = final["err"].mean(axis=0)  # (S,B) normalized MSE of weights
    nmse_sem  = _sem(final["err"])         # (S,B)
    nmse_p10  = np.percentile(final["err"], 10, axis=0)  # (S,B)
    nmse_p90  = np.percentile(final["err"], 90, axis=0)  # (S,B)

    # Make header labels safe for CSV/pgfplots
    safe_names = [_sanitize_baseline_name_for_csv(b.name) for b in baselines]
    orig_names = [b.name for b in baselines]  # for reference/legends

    # 1) SHD CSV
    shd_csv = out_dir / "shd_vs_samples.csv"
    with open(shd_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_mean", f"{sn}_sem", f"{sn}_p10", f"{sn}_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(shd_mean[si, bi]), float(shd_sem[si, bi]),
                        float(shd_p10[si, bi]), float(shd_p90[si, bi])]
            w.writerow(row)

    # 2) NMSE CSV
    nmse_csv = out_dir / "nmse_vs_samples.csv"
    with open(nmse_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_mean", f"{sn}_sem", f"{sn}_p10", f"{sn}_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(nmse_mean[si, bi]), float(nmse_sem[si, bi]),
                        float(nmse_p10[si, bi]), float(nmse_p90[si, bi])]
            w.writerow(row)

    # 3) Wide summary CSV (both metrics interleaved)
    summary_csv = out_dir / "summary_vs_samples.csv"
    with open(summary_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_SHD_mean", f"{sn}_SHD_sem", f"{sn}_SHD_p10", f"{sn}_SHD_p90",
                       f"{sn}_NMSE_mean", f"{sn}_NMSE_sem", f"{sn}_NMSE_p10", f"{sn}_NMSE_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(shd_mean[si, bi]),  float(shd_sem[si, bi]),
                        float(shd_p10[si, bi]),   float(shd_p90[si, bi]),
                        float(nmse_mean[si, bi]), float(nmse_sem[si, bi]),
                        float(nmse_p10[si, bi]),  float(nmse_p90[si, bi])]
            w.writerow(row)

    print("Saved CSVs:", shd_csv, nmse_csv, summary_csv, sep="\n- ")
    return shd_csv, nmse_csv, summary_csv


# save_summary_csvs(final, cfg, out_dir, baselines)

In [6]:
# ---------------------------------------------
# Example: configure and run (10 trials; multi n)
# ---------------------------------------------
# from TopoGreedy import TopoGreedy_refresh_3, TopoGreedy_refresh_2
try:
    _to_jsonable
except NameError:
    def _to_jsonable(x):
        try:
            json.dumps(x)
            return x
        except Exception:
            if hasattr(x, "__name__"):  # functions/classes
                return f"{getattr(x, '__module__', '')}.{x.__name__}"
            return str(x)

# ---------------------------------------------
# Data generation parameters        
# ---------------------------------------------
data_p = {
    "graph_type": "er",
    "n_nodes": 20,
    "edges": 4 * 20,
    "edge_type": "weighted",
    "w_range": ((-2,-0.5) ,(0.5, 2.0)),
    "var": 1.0,
    # "standarize": True,  # if you want to standardize data before emp_cov
}

# ---------------------------------------------
Exps = [
    # BaselineSpec(
    #     model=BUILD,
    #     args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.005},
    #     name="BUILD-0.005",
    #     standardize=False,
    #     adapt_lambda=False,
    #     topo_transpose=False,
    #     is_topogreedy_refresh=True
    # ),
    # BaselineSpec(
    #     model=BUILD,
    #     args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.01},
    #     name="BUILD-0.01",
    #     standardize=False,
    #     adapt_lambda=False,
    #     topo_transpose=False,
    #     is_topogreedy_refresh=True
    # ),
    BaselineSpec(
        model=BUILD,
        args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.1},
        name="BUILD-0.02",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False,
        is_topogreedy_refresh=True
    ),
    # BaselineSpec(
    #     model=BUILD,
    #     args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.04},
    #     name="BUILD-0.04",
    #     standardize=False,
    #     adapt_lambda=False,
    #     topo_transpose=False,
    #     is_topogreedy_refresh=True
    # ),
    #########################################################
    BaselineSpec(
        model=colide_ev,
        init={},
        args={"lambda1": 0.05, "T": 4, "s": [1.0, 0.9, 0.8, 0.7],
              "warm_iter": 2e4, "max_iter": 7e4, "lr": 3e-4,
              "disable_tqdm": True},
        name="CoLiDe-Fix",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False
    ),
    BaselineSpec(
        model=DAGMA_linear,
        init={"loss_type": "l2"},
        args={"lambda1": 0.05, "T": 4, "s": [1.0, 0.9, 0.8, 0.8],
              "warm_iter": 2e4, "max_iter": 7e4, "lr": 3e-4,
              "disable_tqdm": True},
        name="DAGMA",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False
    ),
]
# IMPORTANT: multiple sample sizes + 10 trials
cfg = ExperimentConfig(
    n_graphs=10,                                 # mean over 10 trials
    n_nodes=data_p["n_nodes"],
    n_samples_list=[500, 600, 700, 800, 900, 1000], # choose what you like
    edge_threshold=0.5,
    data_params=data_p,
    baselines=Exps,
    out_dir="./exp_results",
    run_tag=None,
    save_intermediate=True,
    seed_offset=0
)

runner = ExperimentRunner(cfg)
t0 = perf_counter()
final, data = runner.run(verbose=True)
t1 = perf_counter()
print(f"\n----- Completed in {(t1 - t0)/60:.3f} minutes -----")

# Create the two plots and save them next to final_results.npz
plot_summary_curves(final, cfg, runner.out_root, cfg.baselines)



=== Graph 1/10 ===
cond: 69035.28868982918
- samples=500, edges≈71


TypeError: compute_norm_sq_err() missing 1 required positional argument: 'norm_W_true'

In [10]:
# ---------------------------------------------
# Example: configure and run (10 trials; multi n)
# ---------------------------------------------
# from TopoGreedy import TopoGreedy_refresh_3, TopoGreedy_refresh_2
try:
    _to_jsonable
except NameError:
    def _to_jsonable(x):
        try:
            json.dumps(x)
            return x
        except Exception:
            if hasattr(x, "__name__"):  # functions/classes
                return f"{getattr(x, '__module__', '')}.{x.__name__}"
            return str(x)

# ---------------------------------------------
# Data generation parameters        
# ---------------------------------------------
data_p = {
    "graph_type": "er",
    "n_nodes": 200,
    "edges": 4 * 200,
    "edge_type": "weighted",
    "w_range": ((-2,-0.5) ,(0.5, 2.0)),
    "var": 1.0,
    # "standarize": True,  # if you want to standardize data before emp_cov
}

# ---------------------------------------------
Exps = [
    BaselineSpec(
        model=BUILD,
        args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.005},
        name="BUILD-0.005",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False,
        is_topogreedy_refresh=True
    ),
    BaselineSpec(
        model=BUILD,
        args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.01},
        name="BUILD-0.01",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False,
        is_topogreedy_refresh=True
    ),
    BaselineSpec(
        model=BUILD,
        args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.02},
        name="BUILD-0.02",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False,
        is_topogreedy_refresh=True
    ),
    BaselineSpec(
        model=BUILD,
        args={"k_list": [data_p["n_nodes"]], "threshold_list": [1e-4], "topo_thr": 0.5, "refresh_every": 0.04},
        name="BUILD-0.04",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False,
        is_topogreedy_refresh=True
    ),
    #########################################################
    BaselineSpec(
        model=colide_ev,
        init={},
        args={"lambda1": 0.05, "T": 4, "s": [1.0, 0.9, 0.8, 0.7],
              "warm_iter": 2e4, "max_iter": 7e4, "lr": 3e-4,
              "disable_tqdm": True},
        name="CoLiDe-Fix",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False
    ),
    BaselineSpec(
        model=DAGMA_linear,
        init={"loss_type": "l2"},
        args={"lambda1": 0.05, "T": 4, "s": [1.0, 0.9, 0.8, 0.8],
              "warm_iter": 2e4, "max_iter": 7e4, "lr": 3e-4,
              "disable_tqdm": True},
        name="DAGMA",
        standardize=False,
        adapt_lambda=False,
        topo_transpose=False
    ),
]
# IMPORTANT: multiple sample sizes + 10 trials
cfg = ExperimentConfig(
    n_graphs=20,                                 # mean over 10 trials
    n_nodes=data_p["n_nodes"],
    n_samples_list=[1000], # choose what you like
    edge_threshold=0.5,
    data_params=data_p,
    baselines=Exps,
    out_dir="./exp_results",
    run_tag=None,
    save_intermediate=True,
    seed_offset=0
)

runner = ExperimentRunner(cfg)
t0 = perf_counter()
final, data = runner.run(verbose=True)
t1 = perf_counter()
print(f"\n----- Completed in {(t1 - t0)/60:.3f} minutes -----")

# Create the two plots and save them next to final_results.npz
plot_summary_curves(final, cfg, runner.out_root, cfg.baselines)



=== Graph 1/20 ===
cond: 70386725.24646494
- samples=1000, edges≈770


  · BUILD-0.005        | SHD 18.0 | F1 0.988 | ΘΔ 0.307 | 1264.58s
  · BUILD-0.01         | SHD 44.0 | F1 0.972 | ΘΔ 0.307 | 638.75s
  · BUILD-0.02         | SHD 74.0 | F1 0.952 | ΘΔ 0.307 | 331.31s
  · BUILD-0.04         | SHD 114.0 | F1 0.927 | ΘΔ 0.307 | 187.51s
  · CoLiDe-Fix         | SHD 103.0 | F1 0.929 | ΘΔ 0.000 | 100.60s
  · DAGMA              | SHD 125.0 | F1 0.914 | ΘΔ 0.000 | 88.80s

=== Graph 2/20 ===
cond: 34439038.30595772
- samples=1000, edges≈783
  · BUILD-0.005        | SHD 15.0 | F1 0.990 | ΘΔ 0.312 | 1269.68s
  · BUILD-0.01         | SHD 33.0 | F1 0.979 | ΘΔ 0.312 | 664.25s
  · BUILD-0.02         | SHD 107.0 | F1 0.933 | ΘΔ 0.312 | 341.24s
  · BUILD-0.04         | SHD 117.0 | F1 0.927 | ΘΔ 0.312 | 181.06s
  · CoLiDe-Fix         | SHD 99.0 | F1 0.932 | ΘΔ 0.000 | 80.99s
  · DAGMA              | SHD 104.0 | F1 0.929 | ΘΔ 0.000 | 71.03s

=== Graph 3/20 ===
cond: 257115634.60239965
- samples=1000, edges≈775
  · BUILD-0.005        | SHD 12.0 | F1 0.992 | ΘΔ 0.304 | 1163

KeyboardInterrupt: 

In [13]:
# Load and examine the npz file
import numpy as np

# data = np.load('/Users/hamedajorlou/Documents/GROW/exp_results/20250917_004018_f9a912/graph_000/samples_1000/block.npz')


import re

def _sanitize_baseline_name_for_csv(name: str) -> str:
    """Turn 'TopoGreedy-0.02' -> 'TopoGreedy_0_02' (safe CSV column header)."""
    return re.sub(r'[^A-Za-z0-9]+', '_', name).strip('_')

def save_summary_csvs(final: Dict[str, np.ndarray],
                      cfg: ExperimentConfig,
                      out_dir: Path,
                      baselines: List[BaselineSpec]) -> Tuple[Path, Path, Path]:
    """
    Writes three CSVs into out_dir:
      - shd_vs_samples.csv:   n_samples + (mean, sem, p10, p90) per baseline
      - nmse_vs_samples.csv:  n_samples + (mean, sem, p10, p90) per baseline
      - summary_vs_samples.csv: wide table with both metrics interleaved
    Returns the three paths.
    """
    import csv
    n_samples = np.asarray(cfg.n_samples_list)
    G = int(final["shd"].shape[0])
    B = int(final["shd"].shape[2])

    # Means, SEM, and percentiles across graphs
    def _sem(arr):
        return (arr.std(axis=0, ddof=1) / np.sqrt(G)) if G > 1 else np.zeros_like(arr.mean(axis=0))

    shd_mean = final["shd"].mean(axis=0)   # (S,B)
    shd_sem  = _sem(final["shd"])          # (S,B)
    shd_p10  = np.percentile(final["shd"], 10, axis=0)  # (S,B)
    shd_p90  = np.percentile(final["shd"], 90, axis=0)  # (S,B)
    
    nmse_mean = final["err"].mean(axis=0)  # (S,B) normalized MSE of weights
    nmse_sem  = _sem(final["err"])         # (S,B)
    nmse_p10  = np.percentile(final["err"], 10, axis=0)  # (S,B)
    nmse_p90  = np.percentile(final["err"], 90, axis=0)  # (S,B)

    # Make header labels safe for CSV/pgfplots
    safe_names = [_sanitize_baseline_name_for_csv(b.name) for b in baselines]
    orig_names = [b.name for b in baselines]  # for reference/legends

    # 1) SHD CSV
    shd_csv = out_dir / "shd_vs_samples.csv"
    with open(shd_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_mean", f"{sn}_sem", f"{sn}_p10", f"{sn}_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(shd_mean[si, bi]), float(shd_sem[si, bi]),
                        float(shd_p10[si, bi]), float(shd_p90[si, bi])]
            w.writerow(row)

    # 2) NMSE CSV
    nmse_csv = out_dir / "nmse_vs_samples.csv"
    with open(nmse_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_mean", f"{sn}_sem", f"{sn}_p10", f"{sn}_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(nmse_mean[si, bi]), float(nmse_sem[si, bi]),
                        float(nmse_p10[si, bi]), float(nmse_p90[si, bi])]
            w.writerow(row)

    # 3) Wide summary CSV (both metrics interleaved)
    summary_csv = out_dir / "summary_vs_samples.csv"
    with open(summary_csv, "w", newline="") as f:
        w = csv.writer(f)
        header = ["n_samples"]
        for sn in safe_names:
            header += [f"{sn}_SHD_mean", f"{sn}_SHD_sem", f"{sn}_SHD_p10", f"{sn}_SHD_p90",
                       f"{sn}_NMSE_mean", f"{sn}_NMSE_sem", f"{sn}_NMSE_p10", f"{sn}_NMSE_p90"]
        w.writerow(header)
        for si, n in enumerate(n_samples):
            row = [int(n)]
            for bi in range(B):
                row += [float(shd_mean[si, bi]),  float(shd_sem[si, bi]),
                        float(shd_p10[si, bi]),   float(shd_p90[si, bi]),
                        float(nmse_mean[si, bi]), float(nmse_sem[si, bi]),
                        float(nmse_p10[si, bi]),  float(nmse_p90[si, bi])]
            w.writerow(row)

    print("Saved CSVs:", shd_csv, nmse_csv, summary_csv, sep="\n- ")
    return shd_csv, nmse_csv, summary_csv


# save_summary_csvs(final, cfg, out_dir, baselines)

In [ ]:
# Convert the npz results to CSV format suitable for LaTeX/Overleaf
import os
import numpy as np
import pandas as pd

results_path = "exp_results/20250913_012501_25daf9/final_results.npz"
final = dict(np.load(results_path, allow_pickle=True))

# Assume Exps is available and contains the baseline names in order
baseline_names = [b.name for b in Exps]

# Compute means and stds over axis 0 (n_graphs)
shd_mean = final["shd"].mean(axis=0).squeeze()
shd_std = final["shd"].std(axis=0).squeeze()
fdr_mean = final["fdr"].mean(axis=0).squeeze()
fdr_std = final["fdr"].std(axis=0).squeeze()
tpr_mean = final["tpr"].mean(axis=0).squeeze()
tpr_std = final["tpr"].std(axis=0).squeeze()
f1_mean = final["f1"].mean(axis=0).squeeze()
f1_std = final["f1"].std(axis=0).squeeze()
runtime_mean = final["runtime"].mean(axis=0).squeeze()
runtime_std = final["runtime"].std(axis=0).squeeze()

# Format as "mean ± std" with 2 decimals
def fmt(m, s):
    return [f"{mean:.2f} ± {std:.2f}" for mean, std in zip(m, s)]

df = pd.DataFrame({
    "Baseline": baseline_names,
    "SHD": fmt(shd_mean, shd_std),
    "FDR": fmt(fdr_mean, fdr_std),
    "TPR": fmt(tpr_mean, tpr_std),
    "F1": fmt(f1_mean, f1_std),
    "Time (s)": fmt(runtime_mean, runtime_std)
})

# Display nicely as a table
from IPython.display import display, HTML
display(HTML(df.to_html(index=False, escape=False)))


# Create a LaTeX-friendly summary table
summary_data = []
for b, baseline in enumerate(baseline_names):
    # Format numbers for LaTeX with proper precision
    summary_row = {
        'Baseline': baseline.replace('_', '\\_'),  # Escape underscores for LaTeX
        'SHD': f"{shd_mean[b]:.2f} $\\pm$ {shd_std[b]:.2f}",
        'FDR': f"{fdr_mean[b]:.3f} $\\pm$ {fdr_std[b]:.3f}",
        'TPR': f"{tpr_mean[b]:.3f} $\\pm$ {tpr_std[b]:.3f}",
        'F1': f"{f1_mean[b]:.3f} $\\pm$ {f1_std[b]:.3f}",
        'Time (s)': f"{runtime_mean[b]:.2f} $\\pm$ {runtime_std[b]:.2f}"
    }
    summary_data.append(summary_row)

# Create DataFrame for LaTeX table
latex_df = pd.DataFrame(summary_data)

# Save LaTeX-friendly CSV
latex_csv_path = results_path.replace('.npz', '_latex.csv')
latex_df.to_csv(latex_csv_path, index=False)
print(f"LaTeX-friendly results saved to: {latex_csv_path}")

# Also create a raw data CSV with separate mean/std columns for more flexibility
raw_summary_data = []
for b, baseline in enumerate(baseline_names):
    raw_row = {
        'baseline': baseline,
        'shd_mean': f"{shd_mean[b]:.2f}",
        'shd_std': f"{shd_std[b]:.2f}",
        'fdr_mean': f"{fdr_mean[b]:.3f}",
        'fdr_std': f"{fdr_std[b]:.3f}",
        'tpr_mean': f"{tpr_mean[b]:.3f}",
        'tpr_std': f"{tpr_std[b]:.3f}",
        'f1_mean': f"{f1_mean[b]:.3f}",
        'f1_std': f"{f1_std[b]:.3f}",
        'runtime_mean': f"{runtime_mean[b]:.2f}",
        'runtime_std': f"{runtime_std[b]:.2f}"
    }
    raw_summary_data.append(raw_row)

raw_df = pd.DataFrame(raw_summary_data)
raw_csv_path = results_path.replace('.npz', '_raw_summary.csv')
raw_df.to_csv(raw_csv_path, index=False)
print(f"Raw summary results saved to: {raw_csv_path}")

# Display the LaTeX-ready table
print("\nLaTeX-ready table:")
print(latex_df.to_string(index=False))



ValueError: All arrays must be of the same length